In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import scipy.stats as stats
import yfinance as yf

from mc_simulation import MonteCarloSimulation

In [ ]:
ticker_Symbol = 'URTH'
start = '2010-01-01'
end = '2024-03-01'
num_simulation = 1000
num_days = 252 * 5 # 5 years
initial_investment = 0
saving_per_month = 300

In [ ]:
simu = MonteCarloSimulation(ticker_Symbol, start, end, num_simulation, num_days, initial_investment, saving_per_month)
results = simu.run_simulation()

In [ ]:
# Plot the simulation results using plotly

x_val = np.arange(results.shape[1])
fig = go.Figure()
for i in range(0, num_simulation, 10):
    fig.add_trace(go.Scatter(x=x_val, y=results[i, :], mode='lines', name='Simulation '+str(i)))
fig.add_trace(go.Scatter(x=x_val, y=np.mean(results, axis=0), mode='lines', name='Mean', line=dict(color='black', width=3)))
fig.add_trace(go.Scatter(x=x_val, y=np.percentile(results, 5, axis=0), mode='lines', name='5th Percentile', line=dict(color='red', width=1)))
fig.add_trace(go.Scatter(x=x_val, y=np.percentile(results, 95, axis=0), mode='lines', name='95th Percentile', line=dict(color='red', width=1)))
fig.update_xaxes(tickvals=np.arange(0, num_days, 252), ticktext=np.arange(0, num_days//252))
fig.update_layout(title=f'Monte Carlo Simulation: {ticker_Symbol}', xaxis_title='Years', yaxis_title='Portfolio Sum')
fig.show()


In [ ]:
total_monthly_savings = saving_per_month * num_days//22
total_investment = initial_investment + total_monthly_savings

# Show the distribution of the final portfolio value
print('############# Monte Carlo Simulation Summary ##############')
print('Initial Investment:', initial_investment)
print(f'Total Monthly Savings over {num_days//22} months:', total_monthly_savings)
print(f'Total Investment over {num_days//22} months:', total_investment)

# proportion of simulations that made a profit
profitable_simulations = results[:, -1] > total_investment
print('Proportion of profitable simulations:', np.mean(profitable_simulations), '\n')

final_portfolio_values = results[:, -1]

print('############# Final Portfolio Value Statistics ##############')
print('Mean:', np.mean(final_portfolio_values))
print('Median:', np.median(final_portfolio_values))
print('5th Percentile:', np.percentile(final_portfolio_values, 5))
print('95th Percentile:', np.percentile(final_portfolio_values, 95))
print('Standard Deviation:', np.std(final_portfolio_values))

sns.histplot(final_portfolio_values, kde=True)
plt.xlabel('Portfolio Value')
plt.ylabel('Frequency')

In [ ]:
# subtract the saving_rate for each month and each simulation to calculate returns
saving_rate = saving_per_month
returns = results.copy()
for i in range(num_simulation):
    for j in range(num_days):
        if j % 22 == 0:
            returns[i, j] -= saving_rate
returns

In [ ]:
yearly_returns = pd.DataFrame(returns.T, index=range(num_days+1), columns=[f'simulation_{i}' for i in range(num_simulation)])
yearly_returns.loc[len(yearly_returns)] = yearly_returns.iloc[len(yearly_returns)-1]
print(yearly_returns.shape)
# to compute the yearly returns, we need to compute the cumulative returns for 252 consecutive days for each simulation
yearly_returns = yearly_returns.pct_change().dropna()
yearly_returns = yearly_returns.rolling(252).sum()
yearly_returns = yearly_returns.iloc[::252, :].dropna()
yearly_returns.index.name = 'Year'
yearly_returns.index = (yearly_returns.index + 1)//252

print(yearly_returns.shape)
print(f'Average yearly return: {yearly_returns}')

average_yearly_returns = yearly_returns.mean(axis=0)
print(average_yearly_returns.shape)

In [ ]:
# Show the distribution of the yearly returns
sns.histplot(average_yearly_returns, kde=True)
plt.xlabel('Average yearly returns')
plt.ylabel('Frequency')